In [1]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

# Load data

In [2]:
mydata = np.load('data_ML_Nfreq100_Nbeta20_Nlambda20.npz',allow_pickle=True)
print(mydata.files)

['data', 'freq', 'params']


In [3]:
data = mydata['data']
freq = mydata['freq']
params = mydata['params']

In [4]:
freq.shape

(40000, 256)

In [5]:
data.shape

(40000, 256, 3)

#### same interval

In [6]:

fr=list(freq[0:40000,:])
frT=np.asarray(fr)
#frTB=frT.flatten()
freqBT=np.sort(frT)
freqBT_unique=np.unique(freqBT)

data1R=np.zeros((40000,freqBT_unique.shape[0],3))
data1I=np.zeros((40000,freqBT_unique.shape[0],3))
I=np.arange(0,40000,400)
K=np.arange(256,freqBT_unique.shape[0])
J=np.arange(freqBT_unique.shape[0]-256)
for i,j,k in zip(I,J,K):
    data1R[i:(i+400),j:k,:]=np.real(data[i:(i+400),:,:])
    data1I[i:(i+400),j:k,:]=np.imag(data[i:(i+400),:,:])


In [7]:
# mettre toutes les formes d'ondes  sur la même bande de fréquences
def SameInt(data1R,data1I):
    return np.asarray([u+1j*v for u,v in zip(data1R,data1I)])

In [8]:
data1=SameInt(data1R[:,:,:],data1I[:,:,:])
data1.shape

(40000, 2827, 3)

In [9]:
# verification ( des zéros après la 256 ième fréquence pour FO 0)
data1[0,:,0]

array([1.57502989e-26+6.94253168e-27j, 1.56673787e-26+7.13709609e-27j,
       1.55844074e-26+7.33176966e-27j, ...,
       0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
       0.00000000e+00+0.00000000e+00j])

#### Normalisation

In [11]:
# cette fonction normalise seulement une forme d'onde pour un canal "X","Y","Z"
def normalize(data1,n,e):
# n nombre de FO, e nombre de canal
    
    return np.asarray([data1[n,i,e]/np.linalg.norm(data1[n,:,e]) for i in range(data1.shape[1])])

# return un tableau contenant chaque forme d'onde normalisée

In [124]:
dat1=normalize(data1,10,1)
dat1

array([0.00258263-0.00339323j, 0.00262286-0.00335954j,
       0.00266306-0.00332589j, ..., 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ])

In [199]:
# cette fonction rassemble toutes les formes d'onde normalisée dans un seul tableau 
# n nombre de forme d'onde , e numéro du  canal ("X","Y","Z")
def norm(data1):
    
    return (np.asarray([[normalize(data1,n,e) for e in range (3)] for n in range(400)])).reshape((-1,2827,3))
# pour les n=100 premières formes d'ondes

In [200]:
dat2=norm(data1)
dat2[0,:,0]

array([0.0040876 +0.00180176j, 0.00402299+0.00195334j,
       0.00395815+0.00210546j, ..., 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ])

In [201]:
dat2.shape

(400, 2827, 3)

#### génération de v.a uniforme 

In [202]:
# génération de 5 valeurs de loi uniforme 
x=[np.random.uniform(1,10) for _ in range(5)]
x

[8.734429007727032,
 7.038527146782715,
 5.38758401234275,
 9.615286585123618,
 8.589034204943294]

In [203]:
# multiplication de X par chaque FO
def amplitude(dat2,n,e):
    
    return np.asarray([dat2[n,:,e]*i for i in x])

In [204]:
# cette fonction retourne les 5 amplitudes différentes de chaque FO
def amp(dat2):
    nb_fo=400 # nombre de FO utilisable pour l'apprentissage du réseau de neurones (400)
    return (np.asarray([[amplitude(dat2,n,e)for e in range(3)]for n in range(nb_fo)])).reshape((5,-1,2827,3))
# chaque FO retourne 5 sous formes d'onde d'amplitude différentes chacune ayant 2827 fréquences 



In [205]:
data_final=amp(dat2)

In [206]:
data_final

array([[[[ 0.03570284+0.01573736j,  0.03513854+0.01706135j,
           0.03457218+0.01838998j],
         [ 0.03400221+0.01972691j,  0.03342704+0.02107588j,
           0.03284501+0.02244078j],
         [ 0.03225442+0.02382566j,  0.03165344+0.02523482j,
           0.03104014+0.02667283j],
         ...,
         [ 0.        +0.j        ,  0.        +0.j        ,
           0.        +0.j        ],
         [ 0.        +0.j        ,  0.        +0.j        ,
           0.        +0.j        ],
         [ 0.        +0.j        ,  0.        +0.j        ,
           0.        +0.j        ]],

        [[ 0.03930343+0.01732445j,  0.03868223+0.01878197j,
           0.03805875+0.02024459j],
         [ 0.0374313 +0.02171635j,  0.03679812+0.02320136j,
           0.0361574 +0.0247039j ],
         [ 0.03550725+0.02622845j,  0.03484566+0.02777972j,
           0.0341705 +0.02936276j],
         ...,
         [ 0.        +0.j        ,  0.        +0.j        ,
           0.        +0.j        ],
         [

In [9]:
for i in range(10000):
    
    if np.min(freq[i,:])==np.min(freq[i+1,:]):
        pass
    else:
        freq1=np.min(freq[i,:])
        print("{0},{1}".format(i,freq1))

399,0.002975908915201823
799,0.0029763221740722657
1199,0.0029767354329427085
1599,0.0029771169026692706
1999,0.0029775301615397134
2399,0.002977943420410156
2799,0.0029783248901367188
3199,0.0029787381490071615
3599,0.0029791514078776043
3999,0.002979564666748047
4399,0.002979946136474609
4799,0.002980359395345052
5199,0.0029807726542154947
5599,0.0029811541239420573
5999,0.0029815673828125
6399,0.002981980641682943
6799,0.002982393900553385
7199,0.0029827753702799478
7599,0.0029831886291503905
7999,0.0029836018880208333
8399,0.002983983357747396
8799,0.0029843966166178387
9199,0.0029848098754882814
9599,0.0029852231343587238
9999,0.0029856046040852863


In [10]:
freq1

0.0029856046040852863

# Plot data

In [41]:
def freqIdent(freq):
    N=100 # nombre total de bande
    delta_f=freq[0,1]-freq[0,0]
    nbfreq=freq.shape[1]*N
    l=[np.min(freq[0:400,:])+ i*(delta_f/nbfreq) for i in range(nbfreq)]
    return l
     
        

In [73]:
import pandas as pd
col=['Amplitude','f0(Hz)','fdot','iota(rad)','psi(rad)','phi0(rad)','Ecliptic Latitude (rad)','Ecliptic Longitude (rad)']
params_df=pd.DataFrame(params,columns=col)
pd.set_option('display.max_row',40)
params_df

,Amplitude,f0(Hz),fdot,iota(rad),psi(rad),phi0(rad),Ecliptic Latitude (rad),Ecliptic Longitude (rad)
0,1.000000e-22,0.00298,0.0,0.785398,0.785398,0.0,0.000000,-3.141593
1,1.000000e-22,0.00298,0.0,0.785398,0.785398,0.0,0.000000,-2.810899
2,1.000000e-22,0.00298,0.0,0.785398,0.785398,0.0,0.000000,-2.480205
3,1.000000e-22,0.00298,0.0,0.785398,0.785398,0.0,0.000000,-2.149511
4,1.000000e-22,0.00298,0.0,0.785398,0.785398,0.0,0.000000,-1.818817
...,...,...,...,...,...,...,...,...
39995,1.000000e-22,0.00302,0.0,0.785398,0.785398,0.0,1.570796,1.818817
39996,1.000000e-22,0.00302,0.0,0.785398,0.785398,0.0,1.570796,2.149511
39997,1.000000e-22,0.00302,0.0,0.785398,0.785398,0.0,1.570796,2.480205
39998,1.000000e-22,0.00302,0.0,0.785398,0.785398,0.0,1.570796,2.810899


In [14]:
print('PARAMETRES:')
print('Amplitude:',params[n,0])
print('f0 (Hz):',params[n,1])
print('fdot:',params[n,2])
print('iota (rad):',params[n,3])
print('psi (rad):',params[n,4])
print('phi0 (rad):',params[n,5])
print('Ecliptic Latitude (rad):',params[n,6])#beta
print('Ecliptic Longitude (rad):',params[n,7])#lambda

PARAMETRES:
Amplitude: 1e-22
f0 (Hz): 0.00298
fdot: 0.0
iota (rad): 0.7853981633974483
psi (rad): 0.7853981633974483
phi0 (rad): 0.0
Ecliptic Latitude (rad): 0.0
Ecliptic Longitude (rad): 1.1574288723751867


##### Ajout d'un AWGN pour  avoir un SNR de 20 dB

- on sait que : SNR(dB)= $P_{\rm signal}$(dB) -  $P_{\rm bruit}$(dB)

   - donc $P_{\rm bruit}$(dB)= $P_{\rm signal}$(dB) -  SNR(dB)
   
- ici P correspond à la puissance moyenne 

In [96]:
SNR=20       # SNR en dB
def noise(data,c,e,n):          # c nombre de canaux 'X','Y'et 'Z'  e nombre de formes d'onde, n nombre d'obser
    mean_noise=0
    for i in range(c):
        for j in range(e):
            y=[]
            y=np.append(y,np.mean(data[j,:n,i])) # récupère la moyenne de chaque FO sur le canal i
            data_amp_dB=10*np.log10(y) # conversion en dB 
            noise_amp_dB= data_amp_dB - SNR # amplitude du bruit en dB (ajout à la partie réelle)
            noise_amp_lin=10**(noise_amp_dB/10)
            t1=np.real(noise_amp_lin)
            t2=np.imag(noise_amp_lin)
            for a,b in zip(t1,t2):
                np.random.seed(0)
    # génération d'un AWGN
                noise=np.random.normal(mean_noise,np.sqrt(abs(a+b)),data[j,:n,i].shape)
                data_noise=data[j,:n,i] + noise
                print("canal {0}, forme d'onde {1}, donnée bruitée {2}".format(i,j,data_noise))
    return data_noise